<a href="https://colab.research.google.com/github/panda1835/youtube-hunting/blob/main/01-Data-Collection/01_video-searcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This code extracts URLs by both keyword searching and channel mining.
Ref: https://pytube.io/en/latest/ 
"""

In [1]:
!pip install pytube

     |████████████████████████████████| 56 kB 4.2 MB/s 


In [1]:
!pip install git+https://github.com/mytja/pytube

  Cloning https://github.com/mytja/pytube to /tmp/pip-req-build-p8ojrsie
  Running command git clone -q https://github.com/mytja/pytube /tmp/pip-req-build-p8ojrsie
  Created wheel for pytube: filename=pytube-11.0.1-py3-none-any.whl size=56243 sha256=d85e5a742573fc00e3fe9d91f039953b38138a040b26f6d84d8ca5ea8ed1cf72
  Stored in directory: /tmp/pip-ephem-wheel-cache-d6et0wro/wheels/da/a2/95/308838f31174d736cf6fdf320f6b53ddce0ebc5f8860e32620
Successfully built pytube


In [ ]:
!wget https://raw.githubusercontent.com/panda1835/youtube-hunting/main/Utils/data_collection_utils.py
!wget https://raw.githubusercontent.com/panda1835/youtube-hunting/main/Utils/utils.py

In [3]:
import os
import pytube
from datetime import datetime
import pandas as pd
import pytz
import json
from google.colab import data_table
from time import sleep 
import importlib

import data_collection_utils as dcutils
import utils 
from tqdm import tqdm

In [15]:
importlib.reload(dcutils)
importlib.reload(utils)

<module 'utils' from '/content/utils.py'>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
root = "/content/drive/My Drive/Let-Hunt-Youtube/"
config_file_path = os.path.join(root, 'config.json')

config = json.load(open(config_file_path))

database_save_folder = config['folder']['database']
video_url_db_file_name = config['database']['video_url']
channel_url_db_file_name = config['database']['channel_url']

video_url_file_path = os.path.join(root, database_save_folder, video_url_db_file_name)
channel_url_file_path = os.path.join(root, database_save_folder, channel_url_db_file_name)
video_url_50_file_path = os.path.join(root, database_save_folder, "VIDEO_URL_50.csv")

In [18]:
#@title Display current VIDEO url database
video_url_df = pd.read_csv(video_url_file_path)
print(len(video_url_df))
utils.display_table(video_url_df, data_table)

41


,video_url,added_date,search_query,video_file_name,channel_name,channel_url,extracted_date
0,https://www.youtube.com/watch?v=JJoguqMiNF8,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,Hunter Miền Trung,https://www.youtube.com/channel/UC96UxNOLjm7mX...,NaN
1,https://www.youtube.com/watch?v=4phmj741mLM,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,PCP MIỀN TRUNG HUNTING,https://www.youtube.com/channel/UCvm2fVSraKe4u...,NaN
2,https://www.youtube.com/watch?v=OPQbZd9cg0w,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,Level Pcp VN,https://www.youtube.com/channel/UCS8Jp-2YxlU9k...,NaN
3,https://www.youtube.com/watch?v=6LgVQDZx4F0,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,Khung cành củi 3,https://www.youtube.com/channel/UCUrsu3n_9NmbQ...,NaN
4,https://www.youtube.com/watch?v=2uRznT6OFpg,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,SHOP PCP HUY TUẤN 4,https://www.youtube.com/channel/UC8Wg8_slmNraP...,NaN
5,https://www.youtube.com/watch?v=Bk-lhnyBAnQ,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,Tài Tú pcp 1,https://www.youtube.com/channel/UC2G8vYLLDfOYi...,NaN
6,https://www.youtube.com/watch?v=X2JK0Xy1jvM,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,Bảo Tín PHỤ KIỆN PCP,https://www.youtube.com/channel/UCxg3hBz65ljuu...,NaN
7,https://www.youtube.com/watch?v=AVLI-JnWHHU,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,PCP Tuyền Miền Tây 2,https://www.youtube.com/channel/UCq-6Drny_xs_v...,NaN
8,https://www.youtube.com/watch?v=psBtqAz0J7g,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,PCP MIỀN TRUNG HUNTING,https://www.youtube.com/channel/UCvm2fVSraKe4u...,NaN
9,https://www.youtube.com/watch?v=r_k-qQw4myw,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,QT Săn Ná,https://www.youtube.com/channel/UCodoBOQFGVIu4...,NaN


In [11]:
#@title Display current CHANNEL url database
channel_url_df = pd.read_csv(channel_url_file_path)
print(len(channel_url_df))
utils.display_table(channel_url_df, data_table)

36


,channel_name,channel_url,added_date,exploitation_01
0,Hunter Miền Trung,https://www.youtube.com/channel/UC96UxNOLjm7mX...,2021-12-21 22:13:13,NaN
1,PCP MIỀN TRUNG HUNTING,https://www.youtube.com/channel/UCvm2fVSraKe4u...,2021-12-21 22:13:13,NaN
2,Level Pcp VN,https://www.youtube.com/channel/UCS8Jp-2YxlU9k...,2021-12-21 22:13:13,NaN
3,Khung cành củi 3,https://www.youtube.com/channel/UCUrsu3n_9NmbQ...,2021-12-21 22:13:13,NaN
4,SHOP PCP HUY TUẤN 4,https://www.youtube.com/channel/UC8Wg8_slmNraP...,2021-12-21 22:13:13,NaN
5,Tài Tú pcp 1,https://www.youtube.com/channel/UC2G8vYLLDfOYi...,2021-12-21 22:13:13,NaN
6,Bảo Tín PHỤ KIỆN PCP,https://www.youtube.com/channel/UCxg3hBz65ljuu...,2021-12-21 22:13:13,NaN
7,PCP Tuyền Miền Tây 2,https://www.youtube.com/channel/UCq-6Drny_xs_v...,2021-12-21 22:13:13,NaN
8,QT Săn Ná,https://www.youtube.com/channel/UCodoBOQFGVIu4...,2021-12-21 22:13:13,NaN
9,Núi Rừng Đam Mê pcp 1,https://www.youtube.com/channel/UCXbeEO2j9cAbw...,2021-12-21 22:13:13,NaN


# Exploration

In [45]:
#@title Search VIDEO by keyword
keyword = input('Keyword: ')
new_df = dcutils.search_by_keyword(
    keyword=keyword, 
    url_list=video_url_df.video_url.to_list(), 
    today=utils.what_date_today()
    )
utils.display_table(new_df, data_table)

Keyword: pcp săn sóc


,video_url,extracted_date,search_query,video_file_name,channel_name,channel_url
0,https://www.youtube.com/watch?v=8hsemo5R1lA,2021-12-21 22:02:07,keyword: pcp săn sóc,,SĂN BẮN,https://www.youtube.com/channel/UC5pANGwqwqW_S...
1,https://www.youtube.com/watch?v=7XszWkdPBgg,2021-12-21 22:02:07,keyword: pcp săn sóc,,Thiện Xạ PCP,https://www.youtube.com/channel/UClEWyJhO_MvwT...
2,https://www.youtube.com/watch?v=UsMiUgt79Wc,2021-12-21 22:02:07,keyword: pcp săn sóc,,săn bắn hái lượm vn,https://www.youtube.com/channel/UCkLsGV3zq6xMR...
3,https://www.youtube.com/watch?v=XvYksi8xt5I,2021-12-21 22:02:07,keyword: pcp săn sóc,,shop thu na chuyên nòng T -N,https://www.youtube.com/channel/UCxrGCdtk9-WJU...
4,https://www.youtube.com/watch?v=Sa3d7pmnZ68,2021-12-21 22:02:07,keyword: pcp săn sóc,,HTT PCP,https://www.youtube.com/channel/UCRT--KCaePmiO...
5,https://www.youtube.com/watch?v=8HZtrKKhmws,2021-12-21 22:02:07,keyword: pcp săn sóc,,H'MÔNG PcP,https://www.youtube.com/channel/UClURRROVJyh43...
6,https://www.youtube.com/watch?v=2ywLjAaVHkE,2021-12-21 22:02:07,keyword: pcp săn sóc,,PVN PCP,https://www.youtube.com/channel/UCqYj765zgZ_Yy...
7,https://www.youtube.com/watch?v=kwl7dIVvado,2021-12-21 22:02:07,keyword: pcp săn sóc,,PVN PCP,https://www.youtube.com/channel/UCOAVWMWlga-mu...
8,https://www.youtube.com/watch?v=nGv4hErN09Q,2021-12-21 22:02:07,keyword: pcp săn sóc,,Thợ săn tn,https://www.youtube.com/channel/UCOSDSz058WGdT...
9,https://www.youtube.com/watch?v=KP4Y7ULHLAQ,2021-12-21 22:02:07,keyword: pcp săn sóc,,Thợ Săn hmong,https://www.youtube.com/channel/UC042JKA9RwVRf...


In [68]:
#@title Update VIDEO url database
video_url_df = utils.update_database(video_url_df, new_df)
utils.display_table(video_url_df, data_table)

,video_url,added_date,search_query,video_file_name,channel_name,channel_url,extracted_date
0,https://www.youtube.com/watch?v=JJoguqMiNF8,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,Hunter Miền Trung,https://www.youtube.com/channel/UC96UxNOLjm7mX...,NaN
1,https://www.youtube.com/watch?v=4phmj741mLM,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,PCP MIỀN TRUNG HUNTING,https://www.youtube.com/channel/UCvm2fVSraKe4u...,NaN
2,https://www.youtube.com/watch?v=OPQbZd9cg0w,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,Level Pcp VN,https://www.youtube.com/channel/UCS8Jp-2YxlU9k...,NaN
3,https://www.youtube.com/watch?v=6LgVQDZx4F0,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,Khung cành củi 3,https://www.youtube.com/channel/UCUrsu3n_9NmbQ...,NaN
4,https://www.youtube.com/watch?v=2uRznT6OFpg,2021-12-21 16:23:32,keyword: pcp săn bắn,NaN,SHOP PCP HUY TUẤN 4,https://www.youtube.com/channel/UC8Wg8_slmNraP...,NaN
...,...,...,...,...,...,...,...
629,https://www.youtube.com/watch?v=2vQeQR7xtm0,2021-12-21 22:31:39,channel: Thiện Xạ PCP,,Thiện Xạ PCP,https://www.youtube.com/channel/UClEWyJhO_MvwT...,NaN
630,https://www.youtube.com/watch?v=cYlZ7EcoXj8,2021-12-21 22:31:39,channel: Thiện Xạ PCP,,Thiện Xạ PCP,https://www.youtube.com/channel/UClEWyJhO_MvwT...,NaN
631,https://www.youtube.com/watch?v=qkmYOY7GNhU,2021-12-21 22:31:39,channel: Thiện Xạ PCP,,Thiện Xạ PCP,https://www.youtube.com/channel/UClEWyJhO_MvwT...,NaN
632,https://www.youtube.com/watch?v=7XszWkdPBgg,2021-12-21 22:31:39,channel: Thiện Xạ PCP,,Thiện Xạ PCP,https://www.youtube.com/channel/UClEWyJhO_MvwT...,NaN


In [44]:
#@title Save VIDEO url database
assert video_url_df.video_url.nunique() == len(video_url_df), print("duplicated data")
utils.save_database(video_url_df, video_url_file_path)

Database is saved to /content/drive/My Drive/Let-Hunt-Youtube/Database/VIDEO_URL.csv


# Get channels

In [54]:
temp_new_channel = video_url_df[['channel_name', 'channel_url']].drop_duplicates(keep='first')
assert temp_new_channel.channel_url.nunique() == len(temp_new_channel), print('duplication exists')
temp_new_channel['added_date'] = [utils.what_date_today() for _ in range(len(temp_new_channel))]
temp_new_channel['exploitation_01'] = [None for _ in range(len(temp_new_channel))]
utils.display_table(temp_new_channel, data_table)

,channel_name,channel_url,added_date,exploitation_01
0,Hunter Miền Trung,https://www.youtube.com/channel/UC96UxNOLjm7mX...,2021-12-21 22:13:13,None
1,PCP MIỀN TRUNG HUNTING,https://www.youtube.com/channel/UCvm2fVSraKe4u...,2021-12-21 22:13:13,None
2,Level Pcp VN,https://www.youtube.com/channel/UCS8Jp-2YxlU9k...,2021-12-21 22:13:13,None
3,Khung cành củi 3,https://www.youtube.com/channel/UCUrsu3n_9NmbQ...,2021-12-21 22:13:13,None
4,SHOP PCP HUY TUẤN 4,https://www.youtube.com/channel/UC8Wg8_slmNraP...,2021-12-21 22:13:13,None
5,Tài Tú pcp 1,https://www.youtube.com/channel/UC2G8vYLLDfOYi...,2021-12-21 22:13:13,None
6,Bảo Tín PHỤ KIỆN PCP,https://www.youtube.com/channel/UCxg3hBz65ljuu...,2021-12-21 22:13:13,None
7,PCP Tuyền Miền Tây 2,https://www.youtube.com/channel/UCq-6Drny_xs_v...,2021-12-21 22:13:13,None
9,QT Săn Ná,https://www.youtube.com/channel/UCodoBOQFGVIu4...,2021-12-21 22:13:13,None
10,Núi Rừng Đam Mê pcp 1,https://www.youtube.com/channel/UCXbeEO2j9cAbw...,2021-12-21 22:13:13,None


In [55]:
# update channel
channel_url_df = channel_url_df.append(temp_new_channel, ignore_index=True)
channel_url_df.drop_duplicates(subset=['channel_name', 'channel_url'], keep='first', inplace=True)
assert channel_url_df.channel_url.nunique() == len(channel_url_df), print('duplicated exists')
utils.display_table(channel_url_df, data_table)

,channel_name,channel_url,added_date,exploitation_01
0,Hunter Miền Trung,https://www.youtube.com/channel/UC96UxNOLjm7mX...,2021-12-21 22:13:13,None
1,PCP MIỀN TRUNG HUNTING,https://www.youtube.com/channel/UCvm2fVSraKe4u...,2021-12-21 22:13:13,None
2,Level Pcp VN,https://www.youtube.com/channel/UCS8Jp-2YxlU9k...,2021-12-21 22:13:13,None
3,Khung cành củi 3,https://www.youtube.com/channel/UCUrsu3n_9NmbQ...,2021-12-21 22:13:13,None
4,SHOP PCP HUY TUẤN 4,https://www.youtube.com/channel/UC8Wg8_slmNraP...,2021-12-21 22:13:13,None
5,Tài Tú pcp 1,https://www.youtube.com/channel/UC2G8vYLLDfOYi...,2021-12-21 22:13:13,None
6,Bảo Tín PHỤ KIỆN PCP,https://www.youtube.com/channel/UCxg3hBz65ljuu...,2021-12-21 22:13:13,None
7,PCP Tuyền Miền Tây 2,https://www.youtube.com/channel/UCq-6Drny_xs_v...,2021-12-21 22:13:13,None
8,QT Săn Ná,https://www.youtube.com/channel/UCodoBOQFGVIu4...,2021-12-21 22:13:13,None
9,Núi Rừng Đam Mê pcp 1,https://www.youtube.com/channel/UCXbeEO2j9cAbw...,2021-12-21 22:13:13,None


In [56]:
# save CHANNEL url database
utils.save_database(channel_url_df, channel_url_file_path)

Database is saved to /content/drive/My Drive/Let-Hunt-Youtube/Database/CHANNEL_URL.csv


# Exploitation

In [ ]:
#@title Find channel id of video
video_url = input("Video URL: ")
pytube.YouTube(video_url).channel_url

Video URL: https://www.youtube.com/watch?v=64K5xVk6jIk


'https://www.youtube.com/channel/UC1QuqxGUxWBEXhR0MaY9UzA'

In [ ]:
#@title Search by channel
channel_url = input('Channel URL: ')
new_df = utils.search_by_channel(
    channel_url=channel_url,
    Channel=pytube.Channel, 
    url_list=video_url_df.video_url.to_list(), 
    today=utils.what_date_today()
    )
utils.display_table(new_df, data_table)

Channel URL: https://www.youtube.com/channel/UC1QuqxGUxWBEXhR0MaY9UzA


,video_url,extracted_date,search_query,video_file_name
0,https://www.youtube.com/watch?v=E5HBPTlPWMk,2021-12-20 22:13:21,channel: Việt anh pcp,
1,https://www.youtube.com/watch?v=Fb7SpfwRmeo,2021-12-20 22:13:21,channel: Việt anh pcp,
2,https://www.youtube.com/watch?v=qYeBmg7Dx-s,2021-12-20 22:13:21,channel: Việt anh pcp,
3,https://www.youtube.com/watch?v=kLD13AY4lBY,2021-12-20 22:13:21,channel: Việt anh pcp,
4,https://www.youtube.com/watch?v=ug4HemqwzL0,2021-12-20 22:13:21,channel: Việt anh pcp,
5,https://www.youtube.com/watch?v=l_dSYG0wtW4,2021-12-20 22:13:21,channel: Việt anh pcp,
6,https://www.youtube.com/watch?v=Pv16K2Kj2VQ,2021-12-20 22:13:21,channel: Việt anh pcp,
7,https://www.youtube.com/watch?v=H-X_QJxSGSo,2021-12-20 22:13:21,channel: Việt anh pcp,
8,https://www.youtube.com/watch?v=qXg9M_jG1oU,2021-12-20 22:13:21,channel: Việt anh pcp,
9,https://www.youtube.com/watch?v=I_T1pWUfTts,2021-12-20 22:13:21,channel: Việt anh pcp,


In [ ]:
# Search videos of channel database
for channel_url in tqdm(channel_url_df.channel_url.to_list()):
  new_df = dcutils.search_by_channel(
    channel_url=channel_url,
    url_list=video_url_df.video_url.to_list(), 
    today=utils.what_date_today()
    )
  # update
  video_url_df = utils.update_database(video_url_df, new_df)
  # drop duplications
  video_url_df = video_url_df.drop_duplicates(subset=['video_url'], keep='first')
  sleep(5)

utils.display_table(video_url_df, data_table)

In [28]:
exploitation_01 = [utils.what_date_today() for _ in range(len(channel_url_df))]
channel_url_df['exploitation_01'] = exploitation_01

In [30]:
utils.save_database(channel_url_df, channel_url_file_path)

Database is saved to /content/drive/My Drive/Let-Hunt-Youtube/Database/CHANNEL_URL.csv


In [20]:
# save video
assert video_url_df.video_url.nunique() == len(video_url_df), print("duplicated data")
utils.save_database(video_url_df, video_url_file_path)

Database is saved to /content/drive/My Drive/Let-Hunt-Youtube/Database/VIDEO_URL.csv


# Processing

In [34]:
# Shuffle and pick 50 videos
video_url_50_df = video_url_df.sample(frac=1).head(50)
assert video_url_50_df.video_url.nunique() == len(video_url_50_df), print('duplication exists')
utils.save_database(video_url_50_df, video_url_50_file_path)

Database is saved to /content/drive/My Drive/Let-Hunt-Youtube/Database/VIDEO_URL_50.csv


In [37]:
# extract video id
def extract_id_from_video_url(video_url):
  return video_url.split('v=')[1]

video_url_50_df['video_id'] = video_url_50_df.video_url.apply(extract_id_from_video_url)

In [42]:
# add video name in comment sheets
comment_sheet_name = []
for i in range(len(video_url_50_df)):
  comment_sheet_name.append(f"video{str(i+1).zfill(5)}")
video_url_50_df['comment_sheet_name'] = comment_sheet_name

In [44]:
# comment added date
comment_added_date = []
for i in range(len(video_url_50_df)):
  comment_added_date.append(utils.what_date_today())
video_url_50_df['comment_added_date'] = comment_added_date

In [45]:
assert video_url_50_df.video_url.nunique() == len(video_url_50_df), print('duplication exists')
utils.save_database(video_url_50_df, video_url_50_file_path)
utils.display_table(video_url_50_df, data_table)

Database is saved to /content/drive/My Drive/Let-Hunt-Youtube/Database/VIDEO_URL_50.csv


,video_url,added_date,search_query,video_file_name,channel_name,channel_url,extracted_date,video_id,comment_sheet_name,comment_added_date
3003,https://www.youtube.com/watch?v=0lAyOWJmESk,2021-12-22 09:31:29,channel: Nhật Long PCP,,Nhật Long PCP,https://www.youtube.com/channel/UCZfHBm4IhBMCj...,NaN,0lAyOWJmESk,video00001,2021-12-22 11:26:55
889,https://www.youtube.com/watch?v=wQiJKW13eJ0,2021-12-22 09:29:35,channel: cuộc sống vùng quê pcp,,cuộc sống vùng quê pcp,https://www.youtube.com/channel/UCqkdf0_F1Vxa0...,NaN,wQiJKW13eJ0,video00002,2021-12-22 11:26:55
1800,https://www.youtube.com/watch?v=WcGmh2H9e2c,2021-12-22 09:30:38,channel: Thú Vui Hái Lượm,,Thú Vui Hái Lượm,https://www.youtube.com/channel/UChtmfWyYkhjvS...,NaN,WcGmh2H9e2c,video00003,2021-12-22 11:26:55
2133,https://www.youtube.com/watch?v=vemeEExW6rY,2021-12-22 09:30:56,channel: PCP Duy anh,,PCP Duy anh,https://www.youtube.com/channel/UCiishMm18Stfx...,NaN,vemeEExW6rY,video00004,2021-12-22 11:26:55
2516,https://www.youtube.com/watch?v=jLwI-9KUQGM,2021-12-22 09:30:56,channel: PCP Duy anh,,PCP Duy anh,https://www.youtube.com/channel/UCiishMm18Stfx...,NaN,jLwI-9KUQGM,video00005,2021-12-22 11:26:55
2170,https://www.youtube.com/watch?v=-LrKCJPIDic,2021-12-22 09:30:56,channel: PCP Duy anh,,PCP Duy anh,https://www.youtube.com/channel/UCiishMm18Stfx...,NaN,-LrKCJPIDic,video00006,2021-12-22 11:26:55
2708,https://www.youtube.com/watch?v=WLY0Iyg3dbE,2021-12-22 09:31:10,channel: Tổng Hợp PCP Đoàn Kết,,Tổng Hợp PCP Đoàn Kết,https://www.youtube.com/channel/UCaRNHwNiAwQby...,NaN,WLY0Iyg3dbE,video00007,2021-12-22 11:26:55
565,https://www.youtube.com/watch?v=n1_-gXJuyOo,2021-12-22 09:29:01,channel: Núi Rừng Đam Mê pcp 1,,Núi Rừng Đam Mê pcp 1,https://www.youtube.com/channel/UCXbeEO2j9cAbw...,NaN,n1_-gXJuyOo,video00008,2021-12-22 11:26:55
1862,https://www.youtube.com/watch?v=xcV48yV7Dvk,2021-12-22 09:30:38,channel: Thú Vui Hái Lượm,,Thú Vui Hái Lượm,https://www.youtube.com/channel/UChtmfWyYkhjvS...,NaN,xcV48yV7Dvk,video00009,2021-12-22 11:26:55
2710,https://www.youtube.com/watch?v=8maMTMiOnrE,2021-12-22 09:31:10,channel: Tổng Hợp PCP Đoàn Kết,,Tổng Hợp PCP Đoàn Kết,https://www.youtube.com/channel/UCaRNHwNiAwQby...,NaN,8maMTMiOnrE,video00010,2021-12-22 11:26:55


# Test

1. Search the first 20 videos with keyword "pcp săn bắn"
2. Search the first 20 videos with keyword "pcp săn bắt", pick ones that not duplicate from previous search (14 videos)
3. find the most tags from those 34 videos -> pcp condor (5 times)
4. Search the first 20 videos with keyword "pcp condor", pick ones that not duplicate from previous searches (20 videos)
5. remove channels that not (1) in Vietnam, (2) not authentic contents, (3) not related to/focused on hunting with rifles (12 channels)
6. From 36 unique channels, find all videos of them, get 3081 videos
7. Shuffle and pick 50 sample videos

$\Rightarrow$ not bias by the animals, e.g., birds

Example:
+ PVN PCP	https://www.youtube.com/channel/UCqYj765zgZ_YyPme3GjSX3A
+ PVN PCP	https://www.youtube.com/channel/UCOAVWMWlga-muWFxfvuPA8w
+ Aviary PCP	https://www.youtube.com/channel/UCJ3nA6oOtLM7RWyYZz4l9kA
+ Trình TC	https://www.youtube.com/channel/UCCWGFE9To_kCnr8SYGw3bGA
+ Huỳnh Hải	https://www.youtube.com/channel/UCLqCQ8YutHNcKKZNCWQtTEg
+ Shop săn bẫy Tuấn Lan	https://www.youtube.com/channel/UCHRICXjbfqyE7dXqU6itv0g



In [35]:
a = """
https://www.youtube.com/watch?v=JJoguqMiNF8
https://www.youtube.com/watch?v=4phmj741mLM
https://www.youtube.com/watch?v=OPQbZd9cg0w
https://www.youtube.com/watch?v=-h4xzie0VBM
https://www.youtube.com/watch?v=DzD0Ido_llU
https://www.youtube.com/watch?v=6LgVQDZx4F0
https://www.youtube.com/watch?v=1t30oMlB7dI
https://www.youtube.com/watch?v=2uRznT6OFpg
https://www.youtube.com/watch?v=Bk-lhnyBAnQ
https://www.youtube.com/watch?v=X2JK0Xy1jvM
https://www.youtube.com/watch?v=AVLI-JnWHHU
https://www.youtube.com/watch?v=psBtqAz0J7g
https://www.youtube.com/watch?v=r_k-qQw4myw
https://www.youtube.com/watch?v=QvhBV5lkkZI
https://www.youtube.com/watch?v=hIbCKMxQZSU
https://www.youtube.com/watch?v=Wl7iNRvhipw
https://www.youtube.com/watch?v=PtQMRaj4IH0
https://www.youtube.com/watch?v=hZMY-QSitc4
https://www.youtube.com/watch?v=_fFpVpi8jQc
https://www.youtube.com/watch?v=BlPZosSKIW8
https://www.youtube.com/watch?v=7wdFBjc4mZc
https://www.youtube.com/watch?v=iLtZVw4pisE
https://www.youtube.com/watch?v=50wk4onIHQQ
https://www.youtube.com/watch?v=Zp_0TK93Kn8
https://www.youtube.com/watch?v=9_rkNjGvC5A
https://www.youtube.com/watch?v=5ovh5S6-LzU
https://www.youtube.com/watch?v=IlKxWijPRik
https://www.youtube.com/watch?v=xGWmYAZrWS0
https://www.youtube.com/watch?v=_6dEexbsG6I
https://www.youtube.com/watch?v=gR6IVQO7fLA
https://www.youtube.com/watch?v=SlpwSdpVqsQ
https://www.youtube.com/watch?v=PuX3mIYLTJc
https://www.youtube.com/watch?v=VPyJl_rItTA
https://www.youtube.com/watch?v=Myw5k77Rg9Q
https://www.youtube.com/watch?v=oEuYnjmcfEY
https://www.youtube.com/watch?v=UtEOUxE40kQ
https://www.youtube.com/watch?v=yIimz2p5KEA
https://www.youtube.com/watch?v=Zm5YpOrXssw
https://www.youtube.com/watch?v=afTPTJ7a_Hk
https://www.youtube.com/watch?v=DewYeL3zbWw
https://www.youtube.com/watch?v=Kexqapd1NYM
https://www.youtube.com/watch?v=jOQkHSMiP6M
https://www.youtube.com/watch?v=CEYS0yr1qUk
https://www.youtube.com/watch?v=yeLMA5xZZnw
https://www.youtube.com/watch?v=2dtLOzmtNPc
https://www.youtube.com/watch?v=MP-AjxWbPi4
https://www.youtube.com/watch?v=BEh209GAPAw
https://www.youtube.com/watch?v=24pMY8gVSPg
https://www.youtube.com/watch?v=UJzcekhko7Q
https://www.youtube.com/watch?v=C4FDJBPcg4k
https://www.youtube.com/watch?v=GdrriTdGCwE
https://www.youtube.com/watch?v=mqeommqElTM
https://www.youtube.com/watch?v=WNKSmgflmVM
https://www.youtube.com/watch?v=8kbqllfUZ9g
"""

a.replace('\n', ', ')

', https://www.youtube.com/watch?v=JJoguqMiNF8, https://www.youtube.com/watch?v=4phmj741mLM, https://www.youtube.com/watch?v=OPQbZd9cg0w, https://www.youtube.com/watch?v=-h4xzie0VBM, https://www.youtube.com/watch?v=DzD0Ido_llU, https://www.youtube.com/watch?v=6LgVQDZx4F0, https://www.youtube.com/watch?v=1t30oMlB7dI, https://www.youtube.com/watch?v=2uRznT6OFpg, https://www.youtube.com/watch?v=Bk-lhnyBAnQ, https://www.youtube.com/watch?v=X2JK0Xy1jvM, https://www.youtube.com/watch?v=AVLI-JnWHHU, https://www.youtube.com/watch?v=psBtqAz0J7g, https://www.youtube.com/watch?v=r_k-qQw4myw, https://www.youtube.com/watch?v=QvhBV5lkkZI, https://www.youtube.com/watch?v=hIbCKMxQZSU, https://www.youtube.com/watch?v=Wl7iNRvhipw, https://www.youtube.com/watch?v=PtQMRaj4IH0, https://www.youtube.com/watch?v=hZMY-QSitc4, https://www.youtube.com/watch?v=_fFpVpi8jQc, https://www.youtube.com/watch?v=BlPZosSKIW8, https://www.youtube.com/watch?v=7wdFBjc4mZc, https://www.youtube.com/watch?v=iLtZVw4pisE, https:/